## Bilinear-Biquadratic Chain

$$
H=J\sum_i \cos\theta \left( \boldsymbol S_i\cdot \boldsymbol S_{i+1}\right) + \sin\theta\left(\boldsymbol S_i\cdot \boldsymbol S_{i+1}\right)^2
$$

In [ ]:
using MPSKitModels, MPSKit, TensorKit, TensorOperations, Plots

include("hamiltonian.jl")
include("groundstate.jl")
include("plot_spectrum.jl")

using Logging
disable_logging(LogLevel(Info))

## Entanglement Spectra

Schmidt decomposition:
$$
\ket \Psi = \sum_{\alpha=1}^D\lambda_\alpha \ket \Phi^A_\alpha \otimes \ket \Phi^B_\alpha
$$
The Schmidt coefficients resemble entanglement. High coefficients mean more entanglement whilst $\lambda_\alpha=\{1, 0, 0, ...\}$ means we have a product state.

We can approximate the state by truncating the MPS when the Schmidt coefficients are sufficiently low.

#### Ferromagnetic spin=1:
- product state

In [ ]:
HFM = bilinear_biquadratic_hamiltonian(spin=1, θ=π)
Ψ_HFM = optimize_groundstate(HFM, bond=10, maxiter=500)
spectrum_HFM = entanglement_spectrum(Ψ_HFM)
plot_entanglement(spectrum_HFM)
display(current())

#### AntiFerromagnetic spin=1:
- in SPT phase
- not a product state

In [ ]:
HAFM = bilinear_biquadratic_hamiltonian(spin=1, θ=0)
Ψ_HAFM = optimize_groundstate(HAFM, bond=12, maxiter=500)
spectrum_HAFM = entanglement_spectrum(Ψ_HAFM)
plot_entanglement(spectrum_HAFM)
display(current())

#### AKLT:
- $\theta = \arctan(1/3)$
- exactly represented by $\textrm{bond}=2$

In [ ]:
AKLT = bilinear_biquadratic_hamiltonian(spin=1, θ=atan(1/3))
Ψ_AKLT = optimize_groundstate(AKLT, bond=6, maxiter=500)
spectrum_AKLT = entanglement_spectrum(Ψ_AKLT)
plot_entanglement(spectrum_AKLT)
display(current())

## SPT

Choose a state with a symmetry group $\mathrm G$.

The *Fundamental theorem of MPS* $\implies$ different equivalence classes.

Two states of the same 'class' can be connected via a path of gapped parent Hamiltonians, thus no phase transitions $\implies$ class = phase.

If we travel from one class to another we must cross a phase transition if the symmetry is preserved $\implies$ symmetry protected.

The amount of equivalence classes is equal to the order of the second cohomology group $H^2(\mathrm G, \mathrm U_1)$.

Symmetry has to be represented by a **physical linear rep.** but can be represented by a **virtual projective rep.**


#### AntiFerromagnet spin=1

$H^2(\mathrm{SO}_3, \mathrm U_1)=\mathbb Z_2\implies$ 2 phases:
- Trivial phase: **linear rep.**
- Haldane phase: **projective rep.**

The half integer spin reps. are projective reps. of $\mathrm{SO}_3$. The Haldane phase thus has half integer reps.

The right fixed point $\rho$ is invariant under our virtual symmetry:
$$
X_g\rho X_g^\dagger = \rho \implies \textrm{$\rho$ is proportional to }
$$
The degeneracy of the eigenvalues of $\rho$, which are the Schmidt coefficients, are determined by the irreps. of $X_g$.

## Degeneracies

These degeneracies are always even for our spin=1 $\mathrm{SO}_3$ state as the **virtual projective reps.** are the half integer reps. which have even dimension.

In [ ]:
plot_entanglement(spectrum_HAFM)
annotate!(2 + 1, spectrum_HAFM[2], text("spin=1/2", :red, 10))
annotate!(6 + 1, spectrum_HAFM[6], text("spin=3/2", :red, 10))
annotate!(8 + 1, spectrum_HAFM[8], text("spin=1/2", :red, 10))
display(current())

## Convergence

If we optimize the HAFM for $\textrm{bond}=7$ we get poor convergence, whilst $\textrm{bond}=8$ gets good convergence.

The state is approximated by a direct sum of irreps and cutting one of these irreps in half makes for a harder approximation.

In [ ]:
Ψ1 = optimize_groundstate(HAFM, bond=7)
entanglement_spectrum(Ψ1)

In [ ]:
Ψ2 = optimize_groundstate(HAFM, bond=8)
entanglement_spectrum(Ψ2)

## Approaching AKLT

$\theta\to \arctan(1/3)\approx 0.3218$

For increasing $\theta$ we get an increase in the excitation energy of the magnon. The gap increases and we get overal decreased entanglement. 

At $\theta = \arctan(1/3)$ we get the AKLT ground state which is exactly described by an MPS with $\textrm{bond}=2$.


In [ ]:
θ_AKLT_approach = linrange(0, atan(1/3), 15; endpoint=false)

Ψ_AKLT_list = bilinear_biquadratic_θ_range(θ_AKLT_approach, spin=1, bond=12, maxiter=500)
spectra_AKLT = entanglement_spectra(Ψ_AKLT_list)

plot_spectra(θ_AKLT_approach, spectra_AKLT)
display(current())

We take the Schmidt value $\chi=4$ and see how it changes for each $\theta$:

In [ ]:
plot_schmidt_range(θ_AKLT_approach, spectra_AKLT, 3)
vline!([atan(1/3)], label=nothing)
display(current())

In [ ]:
H = bilinear_biquadratic_hamiltonian(spin=1, θ=0.249*π)
Ψ = optimize_groundstate(H, bond=12, maxiter=500)
spectrum = entanglement_spectrum(Ψ)
plot_entanglement(spectrum)
display(current())

In [ ]:
θ_Nematic_approach = linrange(0, π/4, 20; endpoint=false)

Ψ_Nematic_list = bilinear_biquadratic_θ_range(θ_Nematic_approach, spin=1, bond=12, maxiter=500)
spectra_Nematic = entanglement_spectra(Ψ_Nematic_list)

plot_spectra(θ_Nematic_approach, spectra_Nematic)
display(current())

Notes:
- It has been shown that the Haldane gap is only invariant under physical linear representations (not sure if this is always true for SPT phases)
- The order of the virtual irreps in the SPT phase is just what happens to make the ground state the lowest energy
- Excitation gap of the magnon becomes zero as $\theta \to \pi/4$
- gapped H have exponential correlation decay -> MPS are good to describe these low entanglement states
- gapless H have power law decay